In [1]:
import torch
from collections import Counter
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
import os
from glob import glob
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
#from torchsummary import summary
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn
import numpy as np
import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Data science tools
import numpy as np
import pandas as pd
import os

# Image manipulations
from PIL import Image

from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 14

In [3]:
from torch import Tensor, nn
from torch.nn.functional import interpolate

In [12]:
#!pip install anvil-uplink

     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 51 kB 223 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45230 sha256=14d7d65ea5834d7407cac8325fc5e1d435a2817d28efc1c28da85345d515cb88
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [4]:
import anvil.server
anvil.server.connect("GX22SGC2FDHQTTPZCAII7EOI-QMAS2S5WJ63WLJEI")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [4]:
def imshow_tensor(image, ax=None, title=None):
    """Imshow for Tensor."""

    if ax is None:
        fig, ax = plt.subplots()

    # Set the color channel as the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Reverse the preprocessing steps
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Clip the image pixel values
    image = np.clip(image, 0, 1)

    ax.imshow(image)
    plt.axis('off')

    return ax, image

In [5]:
traindir = f"/content/drive/MyDrive/DATA/train"
valdir = f"/content/drive/MyDrive/DATA/test"

In [6]:
save_file_name = f'resnet50-transfer.pt'
checkpoint_path = f'resnet50-transfer.pth'

# Change to fit hardware
batch_size = 512

# Whether to train on a gpu
train_on_gpu = cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')

# Number of gpus
multi_gpu = False
if train_on_gpu:
    gpu_count = cuda.device_count()
    print(f'{gpu_count} gpus detected.')
    if gpu_count > 1:
        multi_gpu = True
    else:
        multi_gpu = False
    print(train_on_gpu,multi_gpu)

Train on gpu: True
1 gpus detected.
True False


In [28]:
def load_checkpoint(path):
    """Load a PyTorch model checkpoint

    Params
    --------
        path (str): saved model checkpoint. Must start with `model_name-` and end in '.pth'

    Returns
    --------
        None, save the `model` to `path`

    """

    # Get the model name
    model_name = path.split('-')[0]
    assert (model_name in ['vgg16', 'resnet50'
                           ]), "Path must have the correct model name"

    # Load in checkpoint
    checkpoint = torch.load(path)

    if model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
        # Make sure to set parameters as not trainable
        for param in model.parameters():
            param.requires_grad = False
        model.classifier = checkpoint['classifier']

    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
        # Make sure to set parameters as not trainable
        for param in model.parameters():
            param.requires_grad = False
        model.fc = checkpoint['fc']

    # Load in the state dict
    model.load_state_dict(checkpoint['state_dict'])

    total_params = sum(p.numel() for p in model.parameters())
    print(f'{total_params:,} total parameters.')
    total_trainable_params = sum(
        p.numel() for p in model.parameters() if p.requires_grad)
    print(f'{total_trainable_params:,} total gradient parameters.')

    # Move to gpu
    if multi_gpu:
        model = nn.DataParallel(model)

    if train_on_gpu:
        model = model.to('cuda')

    # Model basics
    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_class = checkpoint['idx_to_class']
    model.epochs = checkpoint['epochs']

    # Optimizer
    optimizer = checkpoint['optimizer']
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    return model, optimizer

In [29]:
model, optimizer = load_checkpoint(path=checkpoint_path)

24,041,571 total parameters.
533,539 total gradient parameters.


In [30]:
def process_image(image_path):
    """Process an image path into a PyTorch tensor"""

    image = Image.open(image_path)
    # Resize
    img = image.resize((256, 256))

    # Center crop
    width = 256
    height = 256
    new_width = 224
    new_height = 224

    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    img = img.crop((left, top, right, bottom))

    # Convert to numpy, transpose color dimension and normalize
    img = np.array(img).transpose((2, 0, 1)) / 256

    # Standardization
    means = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
    stds = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))

    img = img - means
    img = img / stds

    img_tensor = torch.Tensor(img)

    return img_tensor

In [31]:
from skimage import measure
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from skimage.metrics import structural_similarity as ssim

In [32]:
#function to compute the mean squared error between images
def mse(imageA, imageB):

	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])

	return err

#function to compute both mse and structural similarity index between two images
def compare_images(imageA, imageB):

	m = mse(imageA, imageB)
	s = ssim(imageA, imageB,multichannel=True)

	return s

In [33]:
def predict(image_path, model, topk=5):
    """Make a prediction for an image using a trained model

    Params
    --------
        image_path (str): filename of the image
        model (PyTorch model): trained model for inference
        topk (int): number of top predictions to return

    Returns
        
    """
    
    real_class = image_path.split('/')[-2]


    # Convert to pytorch tensor
    img_tensor = process_image(image_path)

    # Resize
    if train_on_gpu:
        img_tensor = img_tensor.view(1, 3, 224, 224).cuda()
    else:
        img_tensor = img_tensor.view(1, 3, 224, 224)

    # Set to evaluation
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(img_tensor)
        ps = torch.exp(out)

        # Find the topk predictions
        topk, topclass = ps.topk(topk, dim=1)

        # Extract the actual classes and probabilities
        top_classes = [
            model.idx_to_class[class_] for class_ in topclass.cpu().numpy()[0]
        ]
        top_p = topk.cpu().numpy()[0]

        return img_tensor.cpu().squeeze(), top_p, top_classes, real_class

In [34]:
traindir = f"/content/drive/MyDrive/DATA/train"
valdir = f"/content/drive/MyDrive/DATA/test"

save_file_name = f'resnet50-transfer.pt'
checkpoint_path = f'resnet50-transfer.pth'

save_file_name = f'resnet50-transfer.pt'
checkpoint_path = f'resnet50-transfer.pth'

# Change to fit hardware
batch_size = 512

# Whether to train on a gpu
train_on_gpu = cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')

# Number of gpus
multi_gpu = False
if train_on_gpu:
    gpu_count = cuda.device_count()
    print(f'{gpu_count} gpus detected.')
    if gpu_count > 1:
        multi_gpu = True
    else:
        multi_gpu = False
    print(train_on_gpu,multi_gpu)

Train on gpu: True
1 gpus detected.
True False


In [48]:
import cv2
@anvil.server.callable
def anvil_predict(file):
  print("inside anvil predict")
  # Read the video from specified path
  #cam = cv2.VideoCapture("/content/drive/MyDrive/capstone project/v.mp4")
  print("cam=",type(file))


  with open("tempfilename_video.mp4", 'wb') as cam:
    print(type(cam))
    cam.write(file.get_bytes())

  cam=cv2.VideoCapture("tempfilename_video.mp4")

  currentframe = 0
  cur=None

  while(True):
        
      # reading from frame
      ret,frame = cam.read()

      if ret:

          #no image to compare
          if(cur is None):
            name = 'f' + str(currentframe) + '.jpg'
            print ('Creating...' + name)
            path = '/content/drive/MyDrive/capstone project/frames'
            cv2.imwrite(os.path.join(path,name),frame)
            cur=frame
            currentframe += 1

          else:
            s=compare_images(cur,frame)
            if(s<0.8):
              name = 'f' + str(currentframe) + '.jpg'
              print ('Creating...' + name)
              path = '/content/drive/MyDrive/capstone project/frames'
              cv2.imwrite(os.path.join(path,name),frame)
              cur=frame
              currentframe += 1
        
      else:
          break
    
  # Release all space and windows once done
  cam.release()
  cv2.destroyAllWindows()

  x=0
  res=[]
  #to predict the class of each frame
  stored_images="/content/drive/MyDrive/capstone project/frames"
  for i in range(0,currentframe):

    if(x>currentframe):
      break

    cur="f"+str(x)+".jpg"
    print("cur=",cur)
    cur_path=stored_images+"/"+cur
    img, top_p, top_classes, real_class = predict(cur_path, model,topk=1)
    print(top_p, top_classes)
    cur=cur.replace("f","")
    res.append([cur,top_classes[0]])
    print(res)
    x=x+1
  print(res)
  res=sorted(res,key=lambda x:x[0])
  print(res)
  ans=""

  for i in res:
    ans=ans+i[1]

  print(ans)
  return ans

In [49]:
anvil.server.wait_forever()

inside anvil predict
cam= <class 'anvil._serialise.StreamingMedia'>
<class '_io.BufferedWriter'>
Creating...f0.jpg
Creating...f1.jpg
cur= f0.jpg
[0.9980988] ['H']
[['0.jpg', 'H']]
cur= f1.jpg
[0.99316996] ['I']
[['0.jpg', 'H'], ['1.jpg', 'I']]
[['0.jpg', 'H'], ['1.jpg', 'I']]
[['0.jpg', 'H'], ['1.jpg', 'I']]
HI
inside anvil predict
cam= <class 'anvil._serialise.StreamingMedia'>
<class '_io.BufferedWriter'>
Creating...f0.jpg
Creating...f1.jpg
cur= f0.jpg
[0.9980988] ['H']
[['0.jpg', 'H']]
cur= f1.jpg
[0.99316996] ['I']
[['0.jpg', 'H'], ['1.jpg', 'I']]
[['0.jpg', 'H'], ['1.jpg', 'I']]
[['0.jpg', 'H'], ['1.jpg', 'I']]
HI
inside anvil predict
cam= <class 'anvil._serialise.StreamingMedia'>
<class '_io.BufferedWriter'>
Creating...f0.jpg
Creating...f1.jpg
cur= f0.jpg
[0.9980988] ['H']
[['0.jpg', 'H']]
cur= f1.jpg
[0.99316996] ['I']
[['0.jpg', 'H'], ['1.jpg', 'I']]
[['0.jpg', 'H'], ['1.jpg', 'I']]
[['0.jpg', 'H'], ['1.jpg', 'I']]
HI
inside anvil predict
cam= <class 'anvil._serialise.StreamingM

KeyboardInterrupt: ignored